# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [5]:
# import libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import pandas as pd
from sqlalchemy import create_engine
from sklearn.externals import joblib
import re

from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:

# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('messages_disaster', con=engine)
X = df.message
y = df.drop(['message', 'genre', 'id', 'original'], axis = 1)

### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    '''
    tokenizes a sentence after normalizing it and returns lemmatized tokens.
    '''
    # normalizing, tokenizing, lemmatizing the sentence
    sentence = re.sub('\W',' ',text)
    tokens = word_tokenize(sentence.lower().strip())
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(i,'n') for i in tokens]
    tokens = [lemmatizer.lemmatize(i,'v') for i in tokens]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer = tokenize)),
        ('tfidf',TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
def classification_rep_col(y_test, y_pred):
    for i, col in enumerate(y_test.columns): 
        print('-------:',col,':-------')
        print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

In [7]:
y_pred = pipeline.predict(X_test)
classification_rep_col(y_test, y_pred)

-------: related :-------
             precision    recall  f1-score   support

          0       0.64      0.37      0.47      1582
          1       0.82      0.93      0.87      4923
          2       1.00      0.18      0.31        49

avg / total       0.78      0.79      0.77      6554

-------: request :-------
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5465
          1       0.81      0.43      0.56      1089

avg / total       0.88      0.89      0.87      6554

-------: offer :-------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6530
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      6554

-------: aid_related :-------
             precision    recall  f1-score   support

          0       0.73      0.88      0.80      3875
          1       0.76      0.54      0.63      2679

avg / total       0.75      0.74   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
parameters = {
    'tfidf__norm':['l2','l1'],
    'clf__estimator__min_samples_split':[2,3],
    }

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
cv.fit(X_train,y_train)
y_pred = cv.predict(X_test)
print(cv.best_params_)

{'clf__estimator__min_samples_split': 2, 'tfidf__norm': 'l2'}


In [10]:
accuracy = (y_pred == y_test).sum()/y_test.shape[0]
print('accuracy :', accuracy)
classification_rep_col(y_test, y_pred)

accuracy : related                   0.790204
request                   0.883735
offer                     0.996338
aid_related               0.742142
medical_help              0.920201
medical_products          0.953464
search_and_rescue         0.974367
security                  0.981996
military                  0.967958
child_alone               1.000000
water                     0.959872
food                      0.922338
shelter                   0.931187
clothing                  0.986115
money                     0.979554
missing_people            0.989167
refugees                  0.967348
death                     0.962313
other_aid                 0.870613
infrastructure_related    0.934086
transport                 0.954226
buildings                 0.954379
electricity               0.977418
tools                     0.993744
hospitals                 0.988862
shops                     0.995423
aid_centers               0.989625
other_infrastructure      0.954074
weather_r

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
# using AdaBoost
pipeline_ada = Pipeline([
        ('vect',CountVectorizer(tokenizer = tokenize)),
        ('tfidf',TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators=50, learning_rate = 1)))
])

# using KNeighborsClassifier
pipeline_kneighbors = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', KNeighborsClassifier())
])

In [12]:
pipeline_ada.fit(X_train, y_train)
y_pred = pipeline_ada.predict(X_test)
classification_rep_col(y_test, y_pred)

-------: related :-------
             precision    recall  f1-score   support

          0       0.58      0.26      0.35      1582
          1       0.79      0.94      0.86      4923
          2       0.26      0.10      0.15        49

avg / total       0.74      0.77      0.73      6554

-------: request :-------
             precision    recall  f1-score   support

          0       0.91      0.97      0.94      5465
          1       0.76      0.54      0.63      1089

avg / total       0.89      0.90      0.89      6554

-------: offer :-------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6530
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      6554

-------: aid_related :-------
             precision    recall  f1-score   support

          0       0.76      0.87      0.81      3875
          1       0.77      0.61      0.68      2679

avg / total       0.77      0.77   

In [13]:
pipeline_kneighbors.fit(X_train, y_train)
y_pred = pipeline_kneighbors.predict(X_test)
classification_rep_col(y_test, y_pred)

-------: related :-------
             precision    recall  f1-score   support

          0       0.61      0.31      0.41      1582
          1       0.81      0.93      0.86      4923
          2       0.35      0.39      0.37        49

avg / total       0.75      0.78      0.75      6554

-------: request :-------
             precision    recall  f1-score   support

          0       0.84      1.00      0.91      5465
          1       0.78      0.08      0.15      1089

avg / total       0.83      0.84      0.79      6554

-------: offer :-------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6530
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      6554

-------: aid_related :-------
             precision    recall  f1-score   support

          0       0.60      0.99      0.75      3875
          1       0.76      0.04      0.07      2679

avg / total       0.67      0.60   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Comparing the classification reports of the different models, we choose to go AdaBoostClassifier with as our final model.
These were compared on the basis of the macro average F1 score obtained by all the 3 models. The avg/total F1 scores were compared across all the classes and the highest average F1 score was chosen.
Now we will use gridsearch on AdaBoostClassifier to optimize it.

In [ ]:
parameters = {
    'vect__stop_words': ['english',None],
    'tfidf__use_idf' :[True, False],
    'clf__estimator__n_estimators' : [60,70],
    'clf__estimator__learning_rate' :[1,0.5,2],
    }
cv = GridSearchCV(pipeline_ada, parameters,verbose=2)
cv.fit(X_train, y_train)


Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=english 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=english - 1.4min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=english 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min remaining:    0.0s


[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=english - 1.4min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=english 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=english - 1.4min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=None 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=None - 2.1min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=None 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=None - 2.1min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=None 


In [ ]:
print(cv.best_params_)

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(cv, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.